In [1]:
import pandas as pd
import numpy as np
import math
import os
import codecs

In [2]:
path = os.path.realpath(__name__) # path
drt = os.path.dirname(path)       # directory

In [3]:
finance_import = drt + '/daily/SPI_daily.csv'
articles_import = drt + '/articles_sorted.csv'
ph_import = drt + '/public_holidays_2016-2022.07.csv'

# Import Data

In [4]:
# Import daily SPI data:
fin_dt = pd.read_csv(finance_import)
# Pre-processed articles:
clean_art = pd.read_csv(articles_import, encoding = 'utf-8-sig', sep=';')
# Import 'Public Holidays' data:
ph = pd.read_csv(ph_import)

In [5]:
cols = ['day', 'month','year','log_ret', 'react_label']     # get the columns we need
fin_df = fin_dt.loc[:,cols]                                 # data frame
articles = clean_art.iloc[:,2:9]                            # remove index column, word_count

In [6]:
print(len(fin_df), len(articles))

1636 6082


## Consistent Time-Series

In [7]:
# Load the packages
from datetime import datetime
from datetime import date

The OSE trading times are: Monday through Friday from 9:00 am to 4:20 pm Central European Summer Time (GMT+02:00). The articles are published in GMT, which is 2 hrs before our time zone. The equivalent for the OSE is: Monday through Firday from 7:00 am to 2:20 pm GMT.

To make the articles series consistent with the daily log returns we will group together articles that have been published on each day after 2:20 pm GMT.

Those that fall on Friday, and the weekend have an impact on Monday.

Next, we will check if there are ANY public holidays in the dailly log returns and if there are ANY public holidays in the articles time series. If there are public holidays that fall on a Friday and are inlcuded in the articles series we will check when is the next available return date and make the series consistent.

We get the Weekend Dates for the articles AND the Monthly SPI data:

In [8]:
# First, make the date series:
def get_weekends(info):
    info['date'] = pd.to_datetime({"year": info.year, "month": info.month, "day": info.day})
    info['weekday'] = info['date'].dt.dayofweek
    info['weekend'] = info['weekday'] > 4
    # Filter for ONLY the weekends:
    info_weekends = info[info.weekend == True]
    # Get the dates of the Weekends:
    weekends_dates = pd.to_datetime({"year": info_weekends.year, "month": info_weekends.month, "day": info_weekends.day}).drop_duplicates(keep='first')
    return (info, info.weekend.value_counts(), weekends_dates)

In [9]:
articles_daily, trues_articles, weekends_dates_articles = get_weekends(articles) # 75 dates are weekends
fin_df, trues_fin, weekends_dates_daily = get_weekends(fin_df)                   # No weekends
ph, ph_trues, ph_weekends = get_weekends(ph)                                     # Get the weekends to delete them

In [10]:
# Delete Weekends from public holidays:
ph = ph[(ph.weekend != True)]

Added a new column which we named "impact" and it represents the date on which the article's publication date and time is expected to have an impact on the daily SPI returns:

In [11]:
# Change Fridays after 14:20 GMT to Mondays:
# len(articles_daily.loc[(articles_daily['weekday'] == 4) & (articles_daily['hour'] >=14) & (articles_daily['minute'] >=20)]) # 142
articles_daily.loc[(articles_daily['weekday'] == 4) & (articles_daily['hour'] >=14) & (articles_daily['minute'] >=20), 'impact'] = articles_daily['date'] + pd.to_timedelta(3, unit='D')

# Change Weekends articles to Mondays:
# len(articles_daily.loc[(articles_daily['weekday'] == 5) | (articles_daily['weekday'] == 6)]) # 83
articles_daily.loc[(articles_daily['weekday'] == 5), 'impact'] = articles_daily['date'] + pd.to_timedelta(2, unit='D') # Saturday
articles_daily.loc[(articles_daily['weekday'] == 6), 'impact'] = articles_daily['date'] + pd.to_timedelta(1, unit='D') # Sunday

# Now, all articles published on Monday, Tuesday, Wednesday, and Thursday after 14:20 GMT
# have an impact on next trading day's returns:
articles_daily.loc[(articles_daily['hour'] >= 14) & (articles_daily['minute'] >= 20) & (articles_daily['impact'].isnull()), 'impact'] = articles_daily['date'] + pd.to_timedelta(1, unit='D')

# Also update the remaining as having an impact on the same day returns:
articles_daily.loc[(articles_daily['impact'].isnull()), 'impact'] = articles_daily['date']

Now we repeatedly execute this code until there are no more 'Public Holidays' entries in the 'impact' series. Each iteration checks for the presence of 'ph' and if found, the process is repeated. This continues until the data is completely free of any 'Public Holidays'.

In [12]:
# First, lets check how many of the public holidays are in the 'impact' dates:
articles_daily['ph'] = articles_daily.impact.isin(ph.date).astype(int)
len(articles_daily.loc[articles_daily['ph']==1]) # 157

157

In [13]:
# Separate the half trading public hollidays
ph_half = ph.loc[ph['trading'] == "half"]
# Check if any of them are in the 'impact' column:
articles_daily['ph_half'] = articles_daily.impact.isin(ph_half.date).astype(int)

# We can say that if ph_half == 1, and the time the article was published is before 11:00 GMT then the impact is the date.
articles_daily.loc[(articles_daily['ph_half']==1) & (articles_daily['hour'] <= 10), 'ph'] = 0
# Delete the 'ph_half' column:
articles_daily = articles_daily.iloc[: , :-1]

In [14]:
while True:
    articles_daily['weekday_imp'] = articles_daily['impact'].dt.dayofweek #YES

    # Change them to Monday:
    articles_daily.loc[(articles_daily['ph'] == 1) & (articles_daily['weekday_imp'] == 4), 'impact'] = articles_daily['impact'] + pd.to_timedelta(3, unit='D')
        
    # Check again if those Mondays are also PH:
    articles_daily['ph'] = articles_daily.impact.isin(ph.date).astype(int)
    
    # Change the weekdays again:
    articles_daily['weekday_imp'] = articles_daily['impact'].dt.dayofweek
    
    # Change remaining dates:
    articles_daily.loc[articles_daily['ph'] == 1, 'impact'] = articles_daily['impact'] + pd.to_timedelta(1, unit='D')
    articles_daily['ph'] = articles_daily.impact.isin(ph.date).astype(int)
    
    # Check again how many there are that are Public Holidays:
    if len(articles_daily.loc[articles_daily['ph']== 1]) == 0: # we want this to become 0
        print("DONE")
        break
    else:
        print(f"{len(articles_daily.loc[articles_daily['ph']== 1])} articles left on public holidays")

58 articles left on public holidays
24 articles left on public holidays
DONE


In [15]:
# Delete columns that are not necessary:
articles_daily = articles_daily.drop(["weekday", "weekend", "ph"], axis = 1)

In [16]:
# Get only the dates and compare them with the daily SPI returns:
daily_articles_dates = pd.DataFrame(pd.to_datetime(articles_daily['impact']).drop_duplicates(keep='first'))

# Change column name
daily_articles_dates = daily_articles_dates.rename(columns={"impact":"dates"})

# Save
daily_articles_dates.to_csv('daily/daily_articles_dates.csv', index = False) # without duplicates

In [17]:
print(f"The length of daily articles {len(daily_articles_dates)} and the length of daily SPI returns is {len(fin_df)} and the difference is {len(fin_df) - len(daily_articles_dates)}")

The length of daily articles 1437 and the length of daily SPI returns is 1636 and the difference is 199


We checked and the dates that are on public holiday are the half trading public hollidays.

In [18]:
# Check if there are any public holidays in the trading data:
#fin_df['ph'] = fin_df.date.isin(ph.date).astype(int)
#fin_df['ph_half'] = fin_df.date.isin(ph_half.date).astype(int)
#fin_df = fin_df.iloc[:,:-2] # delete them, we don't need them

In [19]:
# Now we will check which dates in the daily articles also exist in the daily returns:
fin_df['common'] = fin_df['date'].isin(daily_articles_dates.dates).astype(int)
# len(fin_df.loc[fin_df['common'] == 1]) # all 1437 are so all good

In [20]:
# Here we just make sure that all the dates from the daily articles are in the trading dates as well
daily_articles_dates['common'] = daily_articles_dates['dates'].isin(fin_df.date).astype(int)
daily_articles_dates.loc[daily_articles_dates['common']==0]

,dates,common


In [21]:
# Drop unecessary columns:
fin_df = fin_df.drop(["weekday", "weekend"], axis = 1)

# Save:
fin_df.to_csv('daily/daily_common.csv', index = False)

Finally, we are constructing a DataFrame where the dates correspond to the dates of impact, rather than the publication dates.

In [22]:
# Drop some columns to tidy up the data set:
daily_articles_impact = articles_daily.drop(["date", "weekday_imp", "year", "month", "day", "minute", "hour"], axis = 1)
# Rename the column:
daily_articles_impact = daily_articles_impact.rename(columns={"impact":"dates"})

# We are getting year, month, and day for the 'dates' column:
daily_articles_impact['year'] = daily_articles_impact['dates'].dt.year
daily_articles_impact['month'] = daily_articles_impact['dates'].dt.month
daily_articles_impact['day'] = daily_articles_impact['dates'].dt.day

# Sort in ascending order:
daily_articles_impact = daily_articles_impact.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
# Save:
daily_articles_impact.to_csv('articles_daily_ts.csv', encoding = 'utf-8-sig', sep=';', index = False) # with duplicates